In [2]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd

In [3]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[300:] 

In [4]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoCropper(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    '''
    x = sorted(data['x'], reverse = True)
    y = sorted(data['y'], reverse = True)
    
    # Try to exclude outliers
    for i in range(10):
        xMax = round(x[i + 1]) if x[i] - x[i + 1] >= 1 else round(x[i])
        
        xMin = round(x[-i-1]) if x[-i] - x[-i-1] >= 1 else round(x[-i])
            
        yMax = round(y[i + 1]) if y[i] - y[i + 1] >= 1 else round(y[i])

        yMin = round(y[-i-1]) if y[-i] - y[-i-1] >= 1 else round(y[-i])
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
    
    return xMin, yMin, lengthX, lengthY



In [8]:
test = data['x'] - 54
test[test>0]

0       310.710052
1       327.162996
4       357.757609
5       327.016850
10      327.251941
18      326.221389
21      327.006788
22      327.441562
28      327.805285
46      246.855438
48      293.924143
50      358.052064
52      294.117485
54      327.186676
58      327.347165
67      293.905606
68      293.686492
84      246.879775
85      246.672330
90      327.679660
118     247.105536
123     263.720510
124     263.214546
126     247.542611
128     262.610147
134     263.200498
138     255.400768
139     263.306409
144     247.031693
146     246.818972
           ...    
3451     96.839350
3452    101.237265
3453    100.633912
3454     97.695936
3455     98.969257
3456     95.707090
3457     94.629270
3458     90.231025
3459     91.423819
3460     94.270969
3461     95.707768
3462    101.584792
3463    105.665683
3464    116.005055
3465    123.129623
3466    126.512795
3467    134.832328
3468    136.728322
3469    148.136685
3470    153.844775
3471    159.558818
3472    165.

In [5]:
x, y, lengthX, lengthY = getCoord(move_data)

In [6]:
x, y, lengthX, lengthY

(54, 38, 331, 320)

In [83]:
convertX = 440/lengthX
convertY = 440/lengthY
print(convertX, convertY)

1.3293051359516617 1.375


In [80]:
440/331

1.3293051359516617

In [84]:
videoCropper('1034 SI_B, Aug 15, 13 7 49 rotated.mp4', "cropTest rotated.mp4", y, x, lengthY, lengthX)

1034 SI_B, Aug 15, 13 7 49 rotated.mp4 successfully  edited!


In [ ]:
##Question: choose length of the edge in order to be converted in video processing